# Project

## Abstract
We took a look at how the extreme solutions of the free energy equation behave with respect to the temperature and check what happens when we couple the bands with one another. 
We checked that for the first coupling term, $\nu(|\psi_1|^2|\psi_2|^2)$, as when a field is aproximately constant, the coupling term becomes a first order field and it alters direcly the effective temperature, shifting the critical temperature of each band. 
As for the second term, $-\eta(\psi_1\psi_2^*+\psi_2\psi_1^*)$, we checked that depending on the coupling parammeter, $\eta$, the field with the lowest critical temperature, leaves it's usual behavior, when $\eta=0$, and starts approaching the other field's critical temperature.

## Introduction

The free energy, $F$, of a superconductor describes the superconducting transition and can be expressed in terms of a  complex   order parameter  field, $\psi(r) = |\psi(r)|e^{i\phi(r)}$, where the quantity $|\psi(r)|^2$ is a measure of the local density, like a quantum mechanics  wave function <ref name=":0" /> and $\psi(r)$ is nonzero below a phase transition into a superconducting state. The usual free energy equation is expressed as

$$ F = F_n + \alpha |\psi|^2 + \frac{\beta}{2} |\psi|^4 + \frac{1}{2m^*} \left| \left(-i\hbar\nabla - \frac{q^*}{c}\mathbf{A} \right) \psi \right|^2 + \frac{|\mathbf{B}|^2}{8\pi} $$


where $F_n$ is the free energy in the normal phase, $α$ and $β$ in the initial argument were treated as phenomenological parameters, $m$ is an  effective mass (solid-state physics)|effective mass, $e$ is the charge of an electron, $'A$' is the  magnetic vector potential , and $\mathbf{B}=\nabla \times \mathbf{A}$ is the magnetic field. By minimizing the free energy with respect to variations in the order parameter and the vector potential, one arrives at the Ginzburg–Landau equations, which can be useful to determine the field, $\psi$, in terms of the temperature, due to expressing, $\alpha=\alpha_0(T-T_c)$.
In this project, we are going to consider a third order field, to the field $\psi_2$ , and some new coupling terms, where the fields are uniform, and there are no eletric nor magnetic fields, and so the full free energy equation is
$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)+ \frac{\gamma}{3}|\psi_2|^6+\nu(|\psi_1|^2|\psi_2|^2)-\eta(\psi_1\psi_2^*+\psi_2\psi_1^*).$$



## Free energy equation

### Free energy with a third order term

$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)+ \frac{\gamma}{3}|\psi_2|^6$$

Let's minimize the equation by deriving with respect to $\psi^*$, we get,$$\frac{dF}{d\psi^*}=\psi(\alpha+\beta|\psi|^2+\gamma|\psi|^4)=0 \Leftrightarrow |\psi|=\pm\sqrt{\frac{-\beta\pm\sqrt{\beta^2-4\alpha\gamma}}{2\gamma}} \textrm{ V } |\psi|=0,$$

### Free energy with a coupling and without a third order term

#### $F_{\psi_1\psi_2}=-\eta(\psi_1\psi_2^*+\psi_2\psi_1^*)$

$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)-\eta(\psi_1\psi_2^*+\psi_2\psi_1^*)$$

It's painfully hard, to find the extreme solutions of the free energy equation analytically, and so we have to solve it using numerical methods. But even tho we can still simplify this a bit, checking it's derivative with $\phi_1$
$$\frac{\partial F}{\partial \phi}=2|\psi_1||\psi_2|sin(\phi_1-\phi_2)=0 \Leftrightarrow \phi_1=\phi_2$$
and so the coupling term gets reduced gets reduced to
$$F_{\psi_1\psi_2}=-2\eta|\psi_2||\psi_1|$$

#### $F_{\psi_1\psi_2}=\nu(|\psi_1|^2|\psi_2|^2)$

$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)+\nu(|\psi_1|^2|\psi_2|^2)$$

To find the extreme points we take the derivative of the free energy with respect to $\psi_i$,$$\frac{dF}{d\psi_i}=\alpha\psi_i^*+\beta(\psi_i^*)^2\psi_i+\nu|\psi_j|^2\psi_i^*=0 \Leftrightarrow $$
$$\Leftrightarrow \psi_i^*(\alpha+\beta|\psi_i|^2+\nu|\psi_j|^2)=0\Leftrightarrow \psi_i^*=0\vee \alpha+\beta|\psi_i|^2+\nu|\psi_j|^2=0 $$
$$\Leftrightarrow \psi_i=0 \vee |\psi_j|^2=-\frac{1}{\nu}(\alpha+\beta|\psi_i|^2)$$
with $j\neq i$.
$$\left\{\begin{array}{c} |\psi_1|^2=-\frac{1}{\beta}(\alpha+\nu|\psi_2|^2) \\ |\psi_2|^2=-\frac{1}{\beta}(\alpha+\nu|\psi_1|^2)\end{array}\right. \Leftrightarrow |\psi_1|^2=|\psi_2|^2=\frac{-\alpha}{\nu+\beta}$$

### Free energy with coupling

#### $F_{\psi_1\psi_2}=-2\eta|\psi_2||\psi_1|$

$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)+ \frac{\gamma}{3}|\psi_2|^6-2\eta|\psi_2||\psi_1|$$

As mentioned before, and with the new third order term it's even more difficult to determine it, and so we are going to find the extreme solutions, numerically.

#### $F_{\psi_1\psi_2}=\nu(|\psi_1|^2|\psi_2|^2)$

$$F=\alpha_1|\psi_1|^2+\alpha_2|\psi_2|^2+\frac{\beta}{2}(|\psi_1|^4+|\psi_2|^4)+ \frac{\gamma}{3}|\psi_2|^6+\nu(|\psi_1|^2|\psi_2|^2)$$

To find the extreme points we take the derivative of the free energy with respect to {$|\psi_1|,|\psi_2|$},
$$\frac{dF}{d|\psi_1|}=2\alpha|\psi_1|+2\beta|\psi_1|^3+2\nu|\psi_1||\psi_2|^2=0$$
$$\frac{dF}{d|\psi_2|}=2\alpha|\psi_2|+2\beta|\psi_2|^3+2\gamma|\psi_2|^5+2\nu|\psi_1|^2|\psi_2|=0$$

Solving $\psi_1$:

$$\alpha|\psi_1|+\beta|\psi_1|^3+\nu|\psi_1||\psi_2|^2=0 
\Leftrightarrow
|\psi_1|(\alpha+\beta|\psi_1|^2+\nu|\psi_2|^2)=0$$
$$|\psi_1|=0 \vee |\psi_2|^2 = -\frac{1}{\nu}(\alpha+\beta|\psi_1|^2)$$

Solving $\psi_2$:

$$\alpha|\psi_2|+\beta|\psi_2|^3+\nu|\psi_1|^2|\psi_2|+\gamma|\psi_2|^5=0 
\Leftrightarrow
|\psi_2|(\alpha+\beta|\psi_2|^2+\gamma|\psi_2|^4+\nu|\psi_1|^2)=0$$
$$|\psi_2|=0 \vee |\psi_2|^2 = \frac{-\beta\pm\sqrt{\beta^2-4\gamma(\alpha+\nu|\psi_1|^2)}}{2\gamma}$$

Therefore

$$-\frac{1}{\nu}(\alpha+\beta|\psi_1|^2)=\frac{-\beta\pm\sqrt{\beta^2-4\gamma(\alpha+\nu|\psi_1|^2)}}{2\gamma}$$
$$(2\frac{\gamma^2}{\nu^2}\alpha\beta-\frac{\gamma}{\nu}\beta^2+\gamma\nu)|\psi_1|^2 +
(\frac{\gamma^2}{\nu^2}\beta^2)|\psi_1|^4+(\gamma\alpha-\frac{\gamma}{\nu}\beta\alpha+\frac{\gamma^2}{\nu}\alpha^2) = 0$$

$|\psi_1|$ can be expressed as:

$$A = (\frac{\gamma}{\nu}\beta)^2$$
$$B = 2\frac{\gamma^2}{\nu^2}\alpha\beta - \frac{\gamma}{\nu}\beta^2 + \gamma\nu$$
$$C = \frac{\gamma^2}{\nu}\alpha^2+\gamma\alpha-\frac{\gamma}{\nu}\beta\alpha$$
$$|\psi_1|^2 = \frac{-B \pm \sqrt{B^2-4AC}}{2A}$$